In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.cbook as cbook
from progress import print_progress, prepare_print_progress
RE_DELIMITER = r' '

In [2]:
# description of each datafield

# f.eid:            id of a participant
# f.31.0.0          reported sex (phenotype)   0: female, 1: male
# f.22001.0.0       genetic sex                0: female, 1: male
# f.22009.0.1-5     top 5 principle components for genetic data
# f.22021.0.0       kinship

# the number in () is the length of this datafieds reported in the UKbiobank
# the n-m is actually the suffix of a datafield existed in UKB44409.tab file

# f.41200: Operative procedures - main OPCS4 (53)      0-48

# f.41202: Diagnoses - main ICD10   (75)               0-65
# f.41203: Diagnoses - main ICD9    (28)               0-27
# f.41204: Diagnoses - secondary ICD10 (184)           0-183
# f.41205: Diagnoses - secondary ICD9  (30)            0-29

# f.41210: Operative procedures - secondary OPCS4 (97) 0-85
# f.41270: Diagnoses - ICD10        (223)              0-212
# f.41271: Diagnoses - ICD9         (47)               0-46
# f.41272: Operative procedures - OPCS4  (124)         0-116

eid = "f.eid"
sex_n, sex_g = "f.31.0.0", "f.22001.0.0"
pcs = ["f.22009.0.{}".format(i) for i in range(1, 6)]
kinship = "f.22021.0.0"
ethnic = "f.21000.0.0"
main_opcs4s = ["f.41200.0.{}".format(i) for i in range(0, 49)]
main_icd10s = ["f.41202.0.{}".format(i) for i in range(0, 66)]
main_icd9s = ["f.41203.0.{}".format(i) for i in range(0, 28)]
second_icd10s = ["f.41204.0.{}".format(i) for i in range(0, 184)]
second_icd9s = ["f.41205.0.{}".format(i) for i in range(0, 30)]
second_opcs4s = ["f.41210.0.{}".format(i) for i in range(0, 86)]
icd10s = ["f.41270.0.{}".format(i) for i in range(0, 213)]
icd9s = ["f.41271.0.{}".format(i) for i in range(0, 47)]
opcs4s = ["f.41272.0.{}".format(i) for i in range(0, 117)]
distance = "distance-0-0-0"

all_icd9s = icd9s + main_icd9s + second_icd9s
all_icd10s = icd10s + main_icd10s + second_icd10s
all_opcs4s = opcs4s + main_opcs4s + second_opcs4s
all_heart_fields = all_icd9s + all_icd10s + all_opcs4s

data_fields = [eid, sex_n, sex_g, kinship, ethnic]
data_fields = data_fields + pcs + all_heart_fields
data_fields += [distance]

# the index in data_fields is not following the index in .tab file
print(len(data_fields))
# should output 831

831


In [3]:
type_dict = {}
for field in all_heart_fields:
    type_dict[field] = 'str'
    
type_dict[ethnic] = 'int32'

In [4]:
# loading data...
# executing this cell takes time..., be patient. 

file = "data\\phenotypes_all_after_qc.tab"
df = pd.read_csv(file, sep = '\t', dtype = type_dict, low_memory=False)

print("loaded {} rows, and {} columns".format(
    df.shape[0], df.shape[1]))

df.head(10)

loaded 313969 rows, and 831 columns


,f.eid,f.31.0.0,f.21000.0.0,f.22001.0.0,f.22009.0.1,f.22009.0.2,f.22009.0.3,f.22009.0.4,f.22009.0.5,f.22021.0.0,...,f.41272.0.108,f.41272.0.109,f.41272.0.110,f.41272.0.111,f.41272.0.112,f.41272.0.113,f.41272.0.114,f.41272.0.115,f.41272.0.116,distance-0-0-0
0,1000019,1.0,1001,1.0,-10.93920,5.49112,-0.502779,-3.212330,-3.009120,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.250358
1,1000022,0.0,1001,0.0,-11.86800,1.66229,-2.489320,3.538500,-2.192230,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.239663
2,1000054,0.0,1001,0.0,-14.27200,5.29456,-1.102800,5.556450,12.017100,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.262323
3,1000063,0.0,1001,0.0,-11.62040,3.95235,-3.160630,-1.646470,-6.459150,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.674555
4,1000081,1.0,1001,1.0,-5.97240,1.95475,-0.172913,-8.651220,-5.617620,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.286534
5,1000090,1.0,1001,1.0,23.18690,-13.71410,36.499300,-67.851400,2.669810,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.364168
6,1000105,0.0,1001,0.0,-13.46000,5.28783,-0.097231,-0.538677,-5.103140,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.461750
7,1000112,0.0,1001,0.0,-12.28850,6.14563,-1.770760,-0.042921,-0.883904,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.853216
8,1000137,0.0,1001,0.0,-9.48475,5.21706,-1.885400,-0.049029,-1.758830,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.987854
9,1000141,0.0,1001,0.0,-12.05510,3.70951,-1.284980,2.256780,-6.467140,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.678213


In [7]:
# check all heart fields are object(str) instead of int
for i, type_ in enumerate(df.dtypes):
    if str(type_) not in ["object"]:
        print("col index: {}, type: {}".format(i, type_))

col index: 0, type: int64
col index: 1, type: float64
col index: 2, type: int32
col index: 3, type: float64
col index: 4, type: float64
col index: 5, type: float64
col index: 6, type: float64
col index: 7, type: float64
col index: 8, type: float64
col index: 9, type: float64
col index: 830, type: float64


In [8]:
# icd9, icd10, and opcs4 codes for heart diseases
# the following codes were selected from the code file downloaded from
# UKBiobank based on the description in previous cell.

heart_icd9codes = ["410", "4109", "411", "4119", "412", "4129", "42979"]
# we don't have "4101, 4102, 4103, etc., nor 4122, 4123, etc."

heart_icd10codes = ["I21", "I210", "I211", "I212", "I213", "I214", "I219",
                    "I21X", "I22", "I220", "I221", "I228", "I229", "I23",
                    "I230", "I231", "I232", "I233", "I234", "I235", "I236",
                    "I238", "I241", "I252"]

# coronary artery bypass grafting opcs4 codes
cabg_opcs4codes = ["K401", "K402", "K403", "K404", "K411", "K412", 
                   "K413", "K414", "K451", "K452", "K453", "K454", "K455"]

# coronary angioplasty with or without stenting
cas_opcs4codes = ["K491", "K492", "K498", "K499", "K502", "K751", "K752",
                  "K753", "K754", "K758", "K759"]   

heart_opcs4codes = cabg_opcs4codes + cas_opcs4codes

In [9]:
def get_case_df(df):
    """filter case samples(rows) which meet the project requirements.
    A sample is defined to be a case one when either of the following
    conditions is satisfied:
    1. any value in "all_icd9s" datafields can be found in "heart_icd9codes"
    2. any value in "all_icd10s" datafields can be found in "heart_icd10codes"
    3. any value in "all_opcs4s" datafields can be found in "heart_opcs4codes"
    
    params: a dataframe, df
    return: a dataframe which contains only case samples.
    """
    return df[df[all_icd9s].isin(heart_icd9codes).any(axis=1) |
              df[all_icd10s].isin(heart_icd10codes).any(axis=1) |
              df[all_opcs4s].isin(heart_opcs4codes).any(axis=1)]
                  

def get_all_na_df(df):
    """filter samples(rows) which all the heart related datafields are NA.
    the fields are: "all_icd9s" + "all_icd10s" + "all_opcs4s"
    params: a dataframe, df
    return: a dataframe
    """
    return df[df[all_heart_fields].isna().all(axis=1)]

In [10]:
def get_case_proof(row_series, field_code_list):
    """
    find evidence(targeting values) that proves a data row is indeed a case.
    The evidence is a collection containing values in targeting heart related
    codes that exist in heart related datafields.
    params:
        a data row in a dataframe
        a list with tuple as its element, each tuple has 2 list element: 
    the first element is target field names, and the second is target codes 
    that will be checked in the values of target fields
    
    return: a list of tuple, [(datafield, value)]
    """
    proofs = []
    for fields, codes in field_code_list:
        true_s = row_series[fields].isin(codes)
        proof_s = row_series[fields][true_s]
        for index in proof_s.index:
            proofs.append((index, proof_s[index]))
            
    return proofs
        
        
def verify_case_df(case_df, output_file = "data\\proof.txt"):
    """vefiry that each case in a dataframe is a case sample. for each case
    collect the proofs that support it's a case, output proofs for all cases
    to a output file for a later review.
    params:
        case_df: a dataframe
        output_file: path for the proofs
    returns: None
    """
    field_code_list = [(all_icd9s, heart_icd9codes), 
                      (all_icd10s, heart_icd10codes),
                      (all_opcs4s, heart_opcs4codes)]
    prepare_print_progress()
    f = open(output_file, "w")
    i, n_rows = 0, case_df.shape[0]
    for _, row in case_df.iterrows():
        proofs = get_case_proof(row, field_code_list)
        line = "eid[{}], {:0>2} proofs: ".format(row[eid], len(proofs))
        for proof in proofs:
            line += "{}({}) ".format(proof[1], proof[0])
            
        line += "\n"
        f.write(line)
        i += 1
        print_progress(i/n_rows)
        
    f.close()

    
def get_non_na_proof(row_series, target_fields):
    """check whether a row(sample) has non NA values in target fields
    params:
        row_series: a pandas row series
        target_fields: a list of datafield name
    returns:
        a list of tuple that contains any non NA values and the field name.
    """
    proofs = []
    not_nan_s = row_series[target_fields].notna()
    proof_s = row_series[target_fields][not_nan_s]
    for index in proof_s.index:
        proofs.append((index, proof_s[index]))
            
    return proofs


def verify_na_df(nan_df, target_fields = all_heart_fields):
    """check whether there are any samples(rows) in the dataframe non NA 
    values in target datafields. print all rows that has non NA values.
    params:
        case_df: a dataframe
        target_fields: list of field names
    returns: None
    """
    prepare_print_progress()
    i, n_rows = 0, nan_df.shape[0]
    lines = []
    for _, row in nan_df.iterrows():
        proofs = get_non_na_proof(row, target_fields)
        if len(proofs) > 0:
            line = "eid[{}], {:0>2} proofs: ".format(row[eid], len(proofs))
            for proof in proofs:
                line += "{}({}) ".format(proof[1], proof[0])
            
            lines.append(line)
        
        i += 1
        print_progress(i/n_rows)
        
    print("\n{} of {} records passed the verification.".format(
        n_rows - len(lines), n_rows))
    for line in lines:
        print(line)

In [11]:
case_df = get_case_df(df)
print(case_df.shape)   
# 14714 case samples

(14714, 831)


In [12]:
verify_case_df(case_df)

Progress:100.00%; elapsed:  3m33s. complete.                  

In [13]:
case_df.groupby(by=[sex_n]).describe()
# 3281 female, 11433 male;

f.eid                                                    \
            count          mean           std        min        25%   
f.31.0.0                                                              
0.0        3281.0  3.487551e+06  1.434868e+06  1000105.0  2253510.0   
1.0       11433.0  3.518298e+06  1.453123e+06  1000767.0  2255826.0   

                                          f.21000.0.0               ...  \
                50%        75%        max       count         mean  ...   
f.31.0.0                                                            ...   
0.0       3440421.0  4737190.0  6024911.0      3281.0  1001.105151  ...   
1.0       3522049.0  4783599.0  6024410.0     11433.0  1001.088428  ...   

         f.22021.0.0      distance-0-0-0                                 \
                 75%  max          count       mean       std       min   
f.31.0.0                                                                  
0.0              0.0  0.0         3281.0  13.300617  3.004636  1.566967   
1.0              0.0  0.0        11433.0  13.396036  3.253540  1.455638   

                                                      
                25%        50%        75%        max  
f.31.0.0                                              
0.0       12.001441  13.226968  14.410134  49.118131  
1.0       12.035173  13.231065  14.378243  49.390694  

[2 rows x 80 columns]

In [14]:
control_df = get_all_na_df(df)
print(control_df.shape)
# 57229 rows with all_heart_fields are all NA.

(57229, 831)


In [15]:
verify_na_df(control_df)

Progress:100.00%; elapsed:  8m45s. complete.                  
57229 of 57229 records passed the verification.


In [16]:
control_df.groupby(by=[sex_n]).describe()
# 28872 female, 28357 male

f.eid                                                     \
            count          mean           std        min         25%   
f.31.0.0                                                               
0.0       28872.0  3.517431e+06  1.448924e+06  1000112.0  2271253.25   
1.0       28357.0  3.511393e+06  1.449168e+06  1000210.0  2253113.00   

                                          f.21000.0.0               ...  \
                50%        75%        max       count         mean  ...   
f.31.0.0                                                            ...   
0.0       3507637.0  4770458.5  6024731.0     28872.0  1001.148760  ...   
1.0       3522410.0  4759724.0  6024829.0     28357.0  1001.114928  ...   

         f.22021.0.0      distance-0-0-0                                 \
                 75%  max          count       mean       std       min   
f.31.0.0                                                                  
0.0              0.0  0.0        28872.0  13.279502  3.393187  1.444135   
1.0              0.0  0.0        28357.0  13.310434  3.318331  1.362270   

                                                      
                25%        50%        75%        max  
f.31.0.0                                              
0.0       11.870772  13.119556  14.309147  49.976249  
1.0       11.939455  13.147017  14.302432  49.860249  

[2 rows x 80 columns]